In [11]:
import json
import numpy as np
import pandas as pd
import os
import subprocess
# from main import *
from library import *


In [39]:
def get_namess(data):
    names = []
    for d in data:
        match_names = [x['id'] for x in d]
        names.extend(match_names)
    names = list(map(lambda x: x.lower(), names))
    return np.unique(names)

In [41]:
bookie_names = []
#data is an array of json loaded from each file
data = []
filepaths = get_file_paths('data/esports')

for path in filepaths:
    data.append(open_data(path))
    bookie_names.append(path.split('/')[-1].split('.')[0])
    
match_names = get_namess(data)
# match_names = list(map(lambda x: x.lower(), match_names))
print(match_names)

df = pd.DataFrame({'match': match_names})

for d, name in zip(data, bookie_names):
    team1_dict = {x['id'].lower(): x['odds'][0] for x in d if len(x['odds']) > 2}
    team2_dict = {x['id'].lower(): x['odds'][2] for x in d if len(x['odds']) > 2}

    team1 = [team1_dict.get(match) for match in match_names]
    team2 = [team2_dict.get(match) for match in match_names]

    df[f'{name}_team1'] = team1
    df[f'{name}_team2'] = team2 

df

best_odds = find_best_odds(df).copy()
best_odds.dropna(inplace=True)
best_odds

['9 pandas.koi' '9ine.eyeballers' 'all knights.nottingham miedo'
 'argentina (palas).draw' 'arsenal fc (alpha).draw'
 'atletico madrid (ivan).draw' 'atletico madrid (val).draw'
 'austrian force willhaben.eintracht spandau'
 'b8 esports.enterprise esports' 'befive.ankora gaming'
 'bk rog esports.team bds academy' 'bleed esports.fnatic'
 'brazylijski luz.alternate attax' 'carpe diem.limitless'
 'case esports.ramboot club' 'cloud9.rare atom' 'co-op.xld gaming'
 'complexity gaming.og' 'diamant esports.crvena zvezda'
 'diamant esports.zero tenacity' 'espionage.alliance' 'falcons.metizport'
 'fc barcelona (carnage).draw' 'fc copenhagen (tongpo).draw'
 'fc schalke 04.e wie einfach e-sports' 'fluffy aimers.party astronouts'
 'fluxo.team solid' 'france (noah).draw' 'gaimin gladiators.og'
 'gamerlegion.big' 'gameward.solary' 'godsent.zero tenacity'
 'heretics academy.barca esports' 'heroic.ninjas in pyjamas'
 'inglorious.enterprise' 'lag gaming.might' 'limitless.carpe diem'
 'lupus esports.spike

,match,best_team1_odds,best_team1_source,best_team2_odds,best_team2_source
0,9 pandas.koi,1.90,sportsbet_team1,1.90,sportsbet_team2
1,9ine.eyeballers,2.53,sportsbet_team1,1.49,tonybet_team2
2,all knights.nottingham miedo,1.82,sportsbet_team1,1.98,sportsbet_team2
3,argentina (palas).draw,1.82,sportsbet_team1,2.55,sportsbet_team2
4,arsenal fc (alpha).draw,1.96,sportsbet_team1,2.30,sportsbet_team2
...,...,...,...,...,...
65,team spirit.elevate,1.001,TAB_team1,21.00,TAB_team2
66,unicorns of love sexy edition.eintracht frankfurt,1.38,sportsbet_team1,2.91,sportsbet_team2
67,unity esports.fraggerz,1.42,sportsbet_team1,2.72,sportsbet_team2
68,vitality.bee.gentle mates,1.95,sportsbet_team1,1.85,sportsbet_team2
